# Discover, Customize and Access NSIDC DAAC Data

This notebook is based off of the [NSIDC-Data-Access-Notebook](https://github.com/nsidc/NSIDC-Data-Access-Notebook) provided through NSIDC's Github organization. 

Now that we've visualized our study areas, we will first explore data coverage, size, and customization (subsetting, reformatting, reprojection) service availability, and then access those associated files. The `Data access for all datasets` notebook provides the steps needed to subset and download all the data we'll be using in the final `Visualize and Analyze Data`.

___A note on data access options:___
We will be pursuing data discovery and access "programmatically" using Application Programming Interfaces, or APIs. 

*What is an API?* You can think of an API as a middle man between an application or end-use (in this case, us) and a data provider. In this case, the data provider is both the metadata repository housing data information (the Common Metadata Repository, or CMR) and NSIDC as the data distributor. These APIs are generally structured as a URL with a base plus individual key-value-pairs separated by ‘&’.

There are other discovery and access methods available from NSIDC as we walked through in the `Introduction` notebook, including access from the data set landing page 'Download Data' tab (e.g. [ATL07 Data Access](https://nsidc.org/data/atl07?qt-data_set_tabs=1#qt-data_set_tabs)) and [NASA Earthdata Search](https://search.earthdata.nasa.gov/). Programmatic API access is beneficial for those of you who want to incorporate data access into your visualization and analysis workflow. This method is also reproducible and documented to ensure data provenance. 

Here are the steps you will learn in this customize and access notebook:
   
1. Search for data programmatically using the Common Metadata Repository API by time and area of interest.
2. Determine subsetting, reformatting, and reprojection capabilities for our data of interest.
3. Access and customize data using NSIDC's data access and service API.

## Import packages


In [3]:
import requests
import getpass
import json
import tutorial_helper_functions as fn # This is our functions module. We created several functions used in this notebook and the Visualize and Analyze notebook.

## Explore data availability using the Common Metadata Repository 

The Common Metadata Repository (CMR) is a high-performance, high-quality, continuously evolving metadata system that catalogs Earth Science data and associated service metadata records. These metadata records are registered, modified, discovered, and accessed through programmatic interfaces leveraging standard protocols and APIs. Note that not all NSIDC data can be searched at the file level using CMR, particularly those outside of the NASA DAAC program. 

CMR API documentation: https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html

### Select data set and determine version number

Data sets are selected by data set IDs (e.g. ATL07). In the CMR API documentation, a data set ids is referred to as a "short name". These short names are located at the top of each NSIDC data set landing page in gray above the full title. We are using the Python Requests package to access the CMR. This search does not require a password.  

In [58]:
CMR_COLLECTIONS_URL = 'https://cmr.earthdata.nasa.gov/search/collections.json'
response = requests.get(CMR_COLLECTIONS_URL, params={'short_name': 'MOD29'})

Data are then converted to [JSON](https://en.wikipedia.org/wiki/JSON) format; a language independant human-readable open-standard file format.

In [59]:
results = json.loads(response.content)
results

{'feed': {'updated': '2019-12-03T17:41:39.931Z',
  'id': 'https://cmr.earthdata.nasa.gov:443/search/collections.json?short_name=MOD29',
  'title': 'ECHO dataset metadata',
  'entry': [{'boxes': ['-90 -180 90 180'],
    'time_start': '2015-12-06T00:00:00.000Z',
    'dif_ids': ['MOD29_6NRT'],
    'version_id': '6NRT',
    'updated': '2016-02-04T00:00:00.000Z',
    'dataset_id': 'MODIS/Terra Near Real Time (NRT) Sea Ice Extent 5-Min L2 Swath 1km',
    'has_spatial_subsetting': False,
    'has_transforms': False,
    'has_variables': False,
    'data_center': 'LANCEMODIS',
    'short_name': 'MOD29',
    'organizations': ['NASA/GSFC/EOS/ESDIS/LANCEMODIS'],
    'title': 'MODIS/Terra Near Real Time (NRT) Sea Ice Extent 5-Min L2 Swath 1km',
    'coordinate_system': 'CARTESIAN',
    'summary': 'MODIS/Terra Near Real Time (NRT) Sea Ice Extent 5-Min L2 Swath 1km (MOD29) contains the following fields: sea ice by reflectance, sea ice by reflectance pixel quality assurance (QA), ice surface temperat

More than one version can exist for a given data set:

In [60]:
for entry in results['feed']['entry']:
    fn.print_cmr_metadata(entry)

dataset_id: MODIS/Terra Near Real Time (NRT) Sea Ice Extent 5-Min L2 Swath 1km, version_id: 6NRT
dataset_id: MODIS/Terra Sea Ice Extent 5-Min L2 Swath 1km V005, version_id: 5
dataset_id: MODIS/Terra Sea Ice Extent 5-Min L2 Swath 1km V006, version_id: 6


We will specify the most recent version for our remaining data set queries.

### Select time and area of interest

We will create a simple dictionary with our short name, version, time, and area of interest. We'll continue to add to this dictionary as we discover more information about our data set. The bounding box coordinates cover our region of interest over the East Siberian Sea and the temporal range covers March 23, 2019. 

In [61]:
bounding_box = '140,72,153,80' # Bounding Box spatial parameter in decimal degree 'W,S,E,N' format. 

In [62]:
temporal = '2019-03-23T00:00:00Z,2019-03-23T23:59:59Z' # Each date in yyyy-MM-ddTHH:mm:ssZ format; date range in start,end format

Start our data dictionary with our data set, version, and area and time of interest. 

**Note that some version IDs include 3 digits and others include only 1 digit. Make sure to enter this value exactly as reported above.**

In [63]:
data_dict = {'short_name': 'MOD29', 
             'version': '6',
             'bounding_box': bounding_box, 
             'temporal': temporal }

### Determine how many files exist over this time and area of interest, as well as the average size and total volume of those granules

We will use the `granule_info` function to query the CMR granule API. The function prints the number of granules, average size, and total volume of those granules. It returns the granule number value, which we will add to our data dictionary.

In [64]:
gran_num = fn.granule_info(data_dict)
data_dict['gran_num'] = gran_num

There are 13 granules of MOD29 version 6 over my area and time of interest.
The average size of each granule is 2.75 MB and the total size of all 13 granules is 35.70 MB


Note that subsetting, reformatting, or reprojecting can alter the size of the granules if those services are applied to your request.

## Determine the subsetting, reformatting, and reprojection services enabled for your data set of interest.

The NSIDC DAAC supports customization (subsetting, reformatting, reprojection) services on many of our NASA Earthdata mission collections. Let's discover whether or not our data set has these services available using the `print_service_options` function. If services are available, we will also determine the specific service options supported for this data set, which we will then add to our data dictionary. 

### Input Earthdata Login credentials

An Earthdata Login account is required to query data services and to access data from the NSIDC DAAC. If you do not already have an Earthdata Login account, visit http://urs.earthdata.nasa.gov to register. We will input our credentials below, and we'll add our email address to our dictionary for use in our final access request.

In [65]:
uid = 'amy.steiker' # Enter Earthdata Login user name

pswd = getpass.getpass('Earthdata Login password: ') # Input and store Earthdata Login password

email = 'amy.steiker@nsidc.org' # Enter email associated with Earthata Login account

data_dict['email'] = email # Add to data dictionary

Earthdata Login password:  ·········


We now need to create an HTTP session in order to store cookies and pass our credentials to the data service URLs. The capability URL below is what we will query to determine service information. 

In [66]:
# Query service capability URL 
capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{data_dict["short_name"]}.{data_dict["version"]}.xml' 

# Create session to store cookie and pass credentials to capabilities url
session = requests.session() 
s = session.get(capability_url)
response = session.get(s.url,auth=(uid,pswd))
response.raise_for_status() # Raise bad request to check that Earthdata Login credentials were accepted 

This function provides a list of all available services:

In [67]:
fn.print_service_options(data_dict, response)

Services available for MOD29 :

Bounding box subsetting
Variable subsetting
Reformatting to the following options: ['GeoTIFF']
Reprojection to the following options: ['GEOGRAPHIC', 'UNIVERSAL TRANSVERSE MERCATOR', 'POLAR STEREOGRAPHIC', 'SINUSOIDAL']


## Discover data availability on your own for ATL07

Go back to the top under the "Select data set and determine version number" heading. Replace all `MOD29` instances with `ATL07` along with its most recent version number, keeping your time and area of interest the same. How does the data volume and service availability compare to MOD29? 

____

### Populate data dictionary with services of interest

We already added our CMR search keywords to our data dictionary, so now we need to add the service options we want to request. A list of all available service keywords for use with NSIDC's access and service API are available in our [Key-Value-Pair table](https://nsidc.org/support/tool/table-key-value-pair-kvp-operands-subsetting-reformatting-and-reprojection-services), as a part of our [Programmatic access guide](https://nsidc.org/support/how/how-do-i-programmatically-request-data-services). For our ATL07 request, we are interested in bounding box, temporal, and variable subsetting. These options crop the data values to the specified ranges and variables of interest. We will enter those values into our data dictionary below.


__Bounding box subsetting:__ Output files are cropped to the specified bounding box extent.

In [68]:
data_dict['bbox'] = '140,72,153,80' # Just like with the CMR bounding box search parameter, this value is provided in decimal degree 'W,S,E,N' format. 

__Temporal subsetting:__ Output files are cropped to the specified temporal range extent.

In [69]:
data_dict['time'] = '2019-03-23T00:00:00,2019-03-23T23:59:59' # Each date in yyyy-MM-ddTHH:mm:ss format; Date range in start,end format

__Variable subsetting:__ Subsets the data set variable or group of variables. For hierarchical data, all lower level variables are returned if a variable group or subgroup is specified. 

For ATL07, we will use only strong beams since these groups contain higher coverage and resolution due to higher surface returns. According to the user guide, the spacecraft was in the backwards orientation during our day of interest, setting the `gt*l` beams as the strong beams. 

We'll use these primary geolocation, height and quality variables of interest for each of the three strong beams. The following descriptions are provided in the [ATL07 Data Dictionary](https://nsidc.org/sites/nsidc.org/files/technical-references/ATL07-data-dictionary-v001.pdf).

`delta_time`: Number of GPS seconds since the ATLAS SDP epoch. 

`latitude`: Latitude, WGS84, North=+, Lat of segment center

`longitude`: Longitude, WGS84, East=+,Lon of segment center

`height_segment_height`: Mean height from along-track segment fit determined by the sea ice algorithm

`height_segment_confidence`: Confidence level in the surface height estimate based on the number of photons; the background noise rate; and the error
analysis

`height_segment_quality`: Height segment quality flag, 1 is good quality, 0 is bad

`height_segment_surface_error_est`: Error estimate of the surface height (reported in meters)

`height_segment_length_seg`: along-track length of segment containing n_photons_actual

In [70]:
data_dict['coverage'] = '/gt1l/sea_ice_segments/delta_time,\
/gt1l/sea_ice_segments/latitude,\
/gt1l/sea_ice_segments/longitude,\
/gt1l/sea_ice_segments/heights/height_segment_confidence,\
/gt1l/sea_ice_segments/heights/height_segment_height,\
/gt1l/sea_ice_segments/heights/height_segment_quality,\
/gt1l/sea_ice_segments/heights/height_segment_surface_error_est,\
/gt1l/sea_ice_segments/heights/height_segment_length_seg,\
/gt2l/sea_ice_segments/delta_time,\
/gt2l/sea_ice_segments/latitude,\
/gt2l/sea_ice_segments/longitude,\
/gt2l/sea_ice_segments/heights/height_segment_confidence,\
/gt2l/sea_ice_segments/heights/height_segment_height,\
/gt2l/sea_ice_segments/heights/height_segment_quality,\
/gt2l/sea_ice_segments/heights/height_segment_surface_error_est,\
/gt2l/sea_ice_segments/heights/height_segment_length_seg,\
/gt3l/sea_ice_segments/delta_time,\
/gt3l/sea_ice_segments/latitude,\
/gt3l/sea_ice_segments/longitude,\
/gt3l/sea_ice_segments/heights/height_segment_confidence,\
/gt3l/sea_ice_segments/heights/height_segment_height,\
/gt3l/sea_ice_segments/heights/height_segment_quality,\
/gt3l/sea_ice_segments/heights/height_segment_surface_error_est,\
/gt3l/sea_ice_segments/heights/height_segment_length_seg'


### Select data access configurations

The data request can be accessed asynchronously or synchronously. The asynchronous option will allow concurrent requests to be queued and processed as orders. Those requested orders will be delivered to the specified email address, or they can be accessed programmatically as shown below. Synchronous requests will automatically download the data as soon as processing is complete. For this tutorial, we will be selecting the asynchronous method. 

In [71]:
base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request' # Set NSIDC data access base URL

data_dict['request_mode'] = 'async' # Set the request mode to asynchronous

data_dict['page_size'] = 2000 # Set the page size to the maximum of 2000, which equals the number of output files returned

## Create the data request API endpoint 
Programmatic API requests are formatted as HTTPS URLs that contain key-value-pairs specifying the service operations that we specified above. We will first create a string of key-value-pairs from our data dictionary and we'll feed those into our API endpoint. This API endpoint can be executed via command line, a web browser, or in Python below. 

In [72]:
# Create a new param_dict with CMR configuration parameters removed from our data_dict 
param_dict = dict((i, data_dict[i]) for i in data_dict if i!='gran_num' and i!='page_num')

param_string = '&'.join("{!s}={!r}".format(k,v) for (k,v) in param_dict.items()) # Convert param_dict to string
param_string = param_string.replace("'","") # Remove quotes

API_request = f'{base_url}?{param_string}' 
print(API_request) # Print API base URL + request parameters

https://n5eil02u.ecs.nsidc.org/egi/request?short_name=MOD29&version=6&bounding_box=140,72,153,80&temporal=2019-03-23T00:00:00Z,2019-03-23T23:59:59Z&page_size=2000&email=amy.steiker@nsidc.org&bbox=140,72,153,80&time=2019-03-23T00:00:00,2019-03-23T23:59:59&request_mode=async


## Request data

We will now download data using the `request_data` function, which utilizes the Python requests library. Our param_dict and HTTP session will be passed to the function to allow Earthdata Login access. The data will be downloaded directly to this notebook directory in a new Outputs folder. The progress of the order will be reported.

In [73]:
fn.request_data(param_dict,session)

Request HTTP response:  201

Order request URL:  https://n5eil02u.ecs.nsidc.org/egi/request?short_name=MOD29&version=6&bounding_box=140%2C72%2C153%2C80&temporal=2019-03-23T00%3A00%3A00Z%2C2019-03-23T23%3A59%3A59Z&page_size=2000&email=amy.steiker%40nsidc.org&bbox=140%2C72%2C153%2C80&time=2019-03-23T00%3A00%3A00%2C2019-03-23T23%3A59%3A59&request_mode=async

order ID:  5000000431176
status URL:  https://n5eil02u.ecs.nsidc.org/egi/request/5000000431176
HTTP response from order response URL:  201

Initial request status is  pending

Status is not complete. Trying again.
Retry request status is:  processing
Status is not complete. Trying again.
Retry request status is:  complete
Zip download URL:  https://n5eil02u.ecs.nsidc.org/esir/5000000431176.zip
Beginning download of zipped output...
Data request is complete.


### Finally, we will clean up the Output folder by removing individual order folders using the `clean_folder` function.

In [74]:
fn.clean_folder()

To review, we have explored data availability and volume over a region and time of interest, discovered and selected data customization options, constructed API endpoints for our requests, and downloaded data. Let's move on to the analysis portion of the tutorial.